<a href="https://colab.research.google.com/github/m3y54m/ut-nndl-course/blob/master/Q1_SOM_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://mnemstudio.org/neural-networks-kohonen-self-organizing-maps.htm
# https://www.saedsayad.com/clustering_som.htm
# Packages and libraries used in this program
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [0]:
# Load dataset with labels
#raw_data = np.load('Alphabets.npy').astype(float)
#labels = np.load('Alphabet_labels.npy')
alphabets =  np.array(['C','I','O','P','S','U','X','Z'])
#initial_weight = np.load('initial_weight.npy')

In [3]:
from google.colab import files
import io
uploaded = files.upload()
raw_data = np.load(io.BytesIO(uploaded['Alphabets.npy'])).astype(float)
uploaded = files.upload()
labels = np.load(io.BytesIO(uploaded['Alphabet_labels.npy']))
uploaded = files.upload()
initial_weight = np.load(io.BytesIO(uploaded['initial_weight.npy']))

Saving Alphabets.npy to Alphabets.npy


Saving Alphabet_labels.npy to Alphabet_labels.npy


Saving initial_weight.npy to initial_weight.npy


In [0]:
# Number of neurons (1-dimensional)
M = 625

# Dimension of the input patterns
N = raw_data.shape[1]

# Total number of input patterns
P = raw_data.shape[0]

learning_rate = 0.2

R = 0

MAX_EPOCHS = 100

MAX_WEIGHT_DIFF = 0.0001

DECAY_FACTOR = 0.00005

RADIUS_REDUCTION_STEP = 20

np.set_printoptions(threshold=np.nan)

In [0]:
# Normalize input vectors
sc = MinMaxScaler(feature_range = (0, 1))
data = sc.fit_transform(raw_data)

In [6]:
# Step 1: Initialization of each node’s weights with a random number between 0 and 1
#weight = 0.1 * np.random.rand(N,M)
weight = np.copy(initial_weight)
#print("Initial weights:")
#print(np.around(weight, 2))

last_weight = np.copy(weight)

for epoch in range(MAX_EPOCHS):
    print("\r\nEpoch:", epoch)
    print("Learning rate:", np.around(learning_rate, 6))
    print("Neighborhood radius:", R)
    
    # Step 2: Choosing input patterns ordering
    # use normal ordering
    pattern_ordering = np.arange(P)
    # use random ordering
    #np.random.shuffle(pattern_ordering)
    #print("Random input patterns ordering:", pattern_ordering)
    
    progress = 0
    
    # For each input pattern do the steps 3-5
    for p in pattern_ordering:
        
        progress = progress + 1
        
        #print("\r\n\tCurrent pattern index is", p, "and", np.around(progress/P*100, 2), "% of patterns has been processed in Epoch", epoch)
        
        # Step 3: Calculating the Best Matching Unit (BMU)

        # initialize distance vector
        distance_vector = np.zeros(M)
    
        # calculate distance of each weight from each input pattern
        for j in range(M):
            for i in range(N):
                distance_vector[j] = distance_vector[j] + (weight[i,j] - data[p,i])**2
                
        #print("\tdistance_vector", np.around(distance_vector, 2))

        # Step 4: find index j such that distance_vector[j] is a minimum
        min_distance_index = np.argmin(distance_vector)
        
        #print("\tmin_distance_index", min_distance_index)
        
        # Step 5: Update weights for all units j within a specified neighberhood of min_distance_index and for all i
        # calculate neighborhood borders
        begin_j = min_distance_index - R
        if (begin_j < 0):
            begin_j = 0
        
        end_j = min_distance_index + R
        if (end_j > M - 1):
            end_j = M - 1
            
        #print("neighberhood", begin_j, end_j)
        
        for j in range(begin_j, end_j + 1):
            for i in range(N):
                weight[i,j] = weight[i,j] + learning_rate * (data[p,i] - weight[i,j])

    # Step 6: Update learning rate
    learning_rate = DECAY_FACTOR * learning_rate

    # Step 7: Reduce radius of topological neighborhood at specified times
    if (epoch % RADIUS_REDUCTION_STEP):
        if R > 0 :
            R = R - 1
        
    # Step 8: Test stopping condition
    weight_diff = np.amax(np.abs(weight - last_weight))
    
    if (weight_diff < MAX_WEIGHT_DIFF):
        print("Weight change:", weight_diff, "<", MAX_WEIGHT_DIFF)
        print("Stopping condition is satisfied!")
        break
    else:
        print("Weight change:", weight_diff, ">", MAX_WEIGHT_DIFF)
    
    last_weight = np.copy(weight)
    # end of for loop
    
#print("\r\Final weights:")
#print(np.around(weight, 2))


Epoch: 0
Learning rate: 0.2
Neighborhood radius: 0
Weight change: 0.9547342521952211 > 0.0001

Epoch: 1
Learning rate: 1e-05
Neighborhood radius: 0
Weight change: 0.00027588661032229567 > 0.0001

Epoch: 2
Learning rate: 0.0
Neighborhood radius: 0
Weight change: 1.378799774354178e-08 < 0.0001
Stopping condition is satisfied!


In [0]:
# associate pattern indices with clusters numbers
patterns_with_clusters = np.zeros((P, 2), dtype=int)

for p in range(P):

    distance_vector = np.zeros(M)

    # calculate distance of each weight from each input pattern
    for j in range(M):
        for i in range(N):
            distance_vector[j] = distance_vector[j] + (weight[i,j] - data[p,i])**2

    # find index j such that distance_vector[j] is a minimum
    min_distance_index = np.argmin(distance_vector)
    
    # store pattern index 
    patterns_with_clusters[p,0] = p
    # store cluster number associated with pattern 
    patterns_with_clusters[p,1] = min_distance_index

In [9]:
# make an array of clusters size and members
clusters_size = [0 for j in range(M)]
clusters_members = [[] for j in range(M)]
for p in range(P):
    # increment cluster size by 1
    clusters_size[patterns_with_clusters[p,1].astype(int)] = clusters_size[patterns_with_clusters[p,1].astype(int)] + 1
    # append the pattern index to cluster members list
    clusters_members[patterns_with_clusters[p,1].astype(int)].append(p)


clustered_patterns = [clusters_size, clusters_members]

# find 20 largest clusters
largest_clusters = np.asarray(clusters_size).argsort()[-20:][::-1]

print("20 Largest Clusters")
total_of_20 = 0
for j in largest_clusters:
    print("Cluster", j, "\t=>\t", clusters_size[j],"patterns.")
    total_of_20 = total_of_20 + clusters_size[j]
print("\r\nTotal", total_of_20, "patterns are in top 20 clusters.")

20 Largest Clusters
Cluster 510 	=>	 100 patterns.
Cluster 496 	=>	 74 patterns.
Cluster 128 	=>	 73 patterns.
Cluster 374 	=>	 68 patterns.
Cluster 48 	=>	 62 patterns.
Cluster 375 	=>	 57 patterns.
Cluster 143 	=>	 23 patterns.
Cluster 478 	=>	 22 patterns.
Cluster 318 	=>	 20 patterns.
Cluster 513 	=>	 1 patterns.
Cluster 624 	=>	 0 patterns.
Cluster 208 	=>	 0 patterns.
Cluster 207 	=>	 0 patterns.
Cluster 206 	=>	 0 patterns.
Cluster 205 	=>	 0 patterns.
Cluster 203 	=>	 0 patterns.
Cluster 204 	=>	 0 patterns.
Cluster 210 	=>	 0 patterns.
Cluster 202 	=>	 0 patterns.
Cluster 201 	=>	 0 patterns.

Total 500 patterns are in top 20 clusters.


In [10]:
original_data = sc.inverse_transform(data)
clusters_size = clustered_patterns[0]
clusters_members = clustered_patterns[1]
for j in largest_clusters:
    print("Cluster", j, "with", clusters_size[j],"patterns:")
    for p in clusters_members[j]:
        print("\r\n\tPattern", p, " with correct interpretation of", alphabets[labels[p]] ,"is put in cluster", j)
        #plt.imshow(original_data[p].reshape(28,28))
        #plt.show()

Cluster 510 with 100 patterns:

	Pattern 5  with correct interpretation of U is put in cluster 510

	Pattern 15  with correct interpretation of U is put in cluster 510

	Pattern 22  with correct interpretation of U is put in cluster 510

	Pattern 26  with correct interpretation of C is put in cluster 510

	Pattern 36  with correct interpretation of U is put in cluster 510

	Pattern 40  with correct interpretation of O is put in cluster 510

	Pattern 42  with correct interpretation of C is put in cluster 510

	Pattern 43  with correct interpretation of C is put in cluster 510

	Pattern 46  with correct interpretation of O is put in cluster 510

	Pattern 51  with correct interpretation of U is put in cluster 510

	Pattern 54  with correct interpretation of U is put in cluster 510

	Pattern 57  with correct interpretation of C is put in cluster 510

	Pattern 61  with correct interpretation of U is put in cluster 510

	Pattern 64  with correct interpretation of U is put in cluster 510

	Pa

In [0]:
#np.save('initial_weight.npy', initial_weight)
#np.save('final_weight.npy', weight)
#initial_weight = np.load('initial_weight.npy')
#weight = np.load('final_weight.npy')